In [7]:
import os
import pandas as pd

# 存储所有结果的字典
results_dict = {'股票名称': []}

# 不同的起始时间
start_times = [pd.to_datetime('2023-02-06'), pd.to_datetime('2023-02-20'), pd.to_datetime('2023-02-24'),pd.to_datetime('2023-03-06'),pd.to_datetime('2023-03-10'),pd.to_datetime('2023-03-14'),pd.to_datetime('2023-03-19'),pd.to_datetime('2023-03-22'),pd.to_datetime('2023-04-16'),pd.to_datetime('2023-05-03'),pd.to_datetime('2023-05-19'),pd.to_datetime('2023-06-13'),pd.to_datetime('2023-06-18'),pd.to_datetime('2023-07-23'),pd.to_datetime('2023-07-25'),pd.to_datetime('2023-07-30'),pd.to_datetime('2023-08-08'),pd.to_datetime('2023-08-24'),pd.to_datetime('2023-09-15'),pd.to_datetime('2023-09-19'),pd.to_datetime('2023-10-02'),pd.to_datetime('2023-10-07'),pd.to_datetime('2023-10-31'),pd.to_datetime('2023-11-16'),pd.to_datetime('2023-11-18'),pd.to_datetime('2023-12-12'),pd.to_datetime('2023-12-13')]  # 可以添加更多起始时间

# 添加每个起始时间对应的空列
for start_time in start_times:
    results_dict[start_time] = []

# 文件夹路径
folder_path = 'stock'

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # 读取Excel文件
        df = pd.read_csv(os.path.join(folder_path, filename))

        # 将时间列转换为日期时间类型
        df['Date'] = pd.to_datetime(df['Date'])

        # 获取股票名称
        stock_name = filename.split('.')[0]
        results_dict['股票名称'].append(stock_name)

        # 计算每个起始时间的变动百分比
        for start_time in start_times:
            # 筛选出起始时间之后一周的数据
            week_data = df[(df['Date'] >= start_time) & (df['Date'] < start_time + pd.Timedelta(days=7))]

            # 计算起始时间之前一周的数据
            last_week_data = df[(df['Date'] >= start_time - pd.Timedelta(days=7)) & (df['Date'] < start_time)]

            # 计算股票价格的均值
            week_mean = week_data['Close'].mean()
            last_week_mean = last_week_data['Close'].mean()

            # 计算变化百分比
            percentage_change = ((week_mean - last_week_mean) / last_week_mean) * 100

            # 存储变动百分比
            results_dict[start_time].append(percentage_change)

# 将字典转换为DataFrame
results_df = pd.DataFrame(results_dict)

# 打印结果
print(results_df)

# 存入新的Excel表中
results_df.to_excel('output.xlsx', index=False)


     股票名称  2023-02-06 00:00:00  2023-02-20 00:00:00  2023-02-24 00:00:00  \
0    AAPL             2.999835            -3.482126            -2.124335   
1    ABBV             0.750645             0.144091             1.713061   
2     AMT            -2.470393            -6.463823            -3.817681   
3    AMZN            -4.749301            -4.263741            -2.806020   
4     APD            -6.719240            -1.235941             2.772703   
5    AVGO             1.262193            -3.597142             0.969922   
6      BA             0.682011            -4.971128            -2.316773   
7     BAC             0.937188            -3.302153            -1.546018   
8   BRK-B            -0.181313            -2.216428             0.311700   
9     CAT            -0.962586            -3.611434             0.922910   
10  CMCSA            -2.724908            -4.192761            -3.000130   
11    COP            -4.179394            -4.527811             1.024490   
12   COST   

In [ ]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel('output.xlsx')

# 遍历DataFrame，修改数值
for i in range(len(df)):
    for j in range(len(df.columns)):
        cell_value = df.iloc[i, j]
        # 检查是否为数字，如果不是数字，则不进行处理
        if isinstance(cell_value, (int, float)):
            if abs(cell_value) >= 5:
                df.iloc[i, j] = abs(cell_value)
            else:
                df.iloc[i, j] = 0

# 将修改后的DataFrame写入新的Excel文件
df.to_excel(r'E:\课题组\个人\论文\多系统风险传导最短路径\network\stock_event.xlsx', index=False)

In [ ]:
import pandas as pd
from datetime import datetime

# 读取 Excel 文件
df = pd.read_excel('stock_event.xlsx')

# 创建一个新的 Excel 文件
with pd.ExcelWriter('stock_event_onmonth.xlsx') as writer:
    # 循环处理每个月份
    for month in range(2, 13):
        # 创建当前月份的 sheet
        sheet_name = f'{month}月'
        # 选择当前月份的数据并将其他月份数据填充为0
        month_df = df.copy()
        for col in month_df.columns[1:]:  # 遍历除了第一列（股票名称）之外的所有列
            # 提取列的月份信息（第一行表示）
            col_month = col.month
            if col_month == month:
                # 如果当前列的月份与目标月份一致，则保留数据，否则置为0
                continue
            else:
                month_df[col] = 0
        # 写入当前月份的数据到新的 sheet
        month_df.to_excel(writer, sheet_name=sheet_name, index=False)
